In [1]:
!pip install fiona

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 53.8 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pathlib
from osgeo import ogr

### Fix shapefiles (agregar campo y valores, reproyectar, guardar en otro formato)

In [4]:
def fix_shapefile(location):
    new_field_name = "clase"
    value = "basural"
    
    target_file = str(pathlib.PosixPath(location).parent) + "/" + str(pathlib.PosixPath(location).stem) + ".gpkg"
    target_epsg = "EPSG:32721"
    
    # Abrir shapefile en modo escritura
    source = ogr.Open(location, update=True)
    layer = source.GetLayer()

    # identificar nombres de columna
    layer_defn = layer.GetLayerDefn()
    field_names = [layer_defn.GetFieldDefn(i).GetName() for i in range(layer_defn.GetFieldCount())]

    # Agregar la columna si no existe

    if new_field_name not in field_names:
        new_field = ogr.FieldDefn(new_field_name, ogr.OFTString)
        layer.CreateField(new_field)
        # Llenar la columna nueva
        feature = layer.GetNextFeature()
        while feature:
            feature.SetField(new_field_name, value)
            layer.SetFeature(feature)
            feature = layer.GetNextFeature()

    # Cerrar el Shapefile
    layer = source = None
    
    # Guardar como geojson y con CRS específico
    !ogr2ogr -f GPKG $target_file -t_srs $target_epsg $location

In [ ]:
%cd "/content/drive/MyDrive/Basurales/AMBA/ENTREGAS - MASCARAS/OCTUBRE 2022/ANOTACIONES"

/content/drive/.shortcut-targets-by-id/1doNB42ZyfxkyNdyA_logHaMoHp7zSpLv/ENTREGAS - MASCARAS/OCTUBRE 2022/ANOTACIONES


In [ ]:
!ls

2015-SEPT   2017-NOV	2019-AGOSTO	2021-ENERO
2016-NOV    2018-ENERO	2019-JULIO	2021-MAYO
2017-DIC    2018-JULIO	2020-DICIEMBRE	2021-NOVIEMBRE
2017-JULIO  2018-NOV	2021-DICIEMBRE	2021-OCTUBRE


In [ ]:
base_dir = "."

for location in pathlib.Path(base_dir).rglob('*.shp'):
    print("Fixing " + location.as_posix())
    fix_shapefile(location.as_posix())

Fixing 2015-SEPT/IMAG-SENT-SEP-2015.shp
Fixing 2017-NOV/IMAG-SENT-NOV-2017.shp
Warning 1: Value of field 'layer' is not a valid UTF-8 string.
Warning 1: Value of field 'path' is not a valid UTF-8 string.
Fixing 2016-NOV/IMAG-SENT-NOV-2016.shp
Fixing 2021-MAYO/IMAG-SENT-MAY-2021.shp
Fixing 2021-ENERO/IMAG-SENT-ENE-2021.shp
Fixing 2018-JULIO/IMAG-SENT-JUL-2018.shp
Fixing 2018-ENERO/IMAG-SENT-ENE-2018.shp
Fixing 2021-OCTUBRE/IMAG-SENT-OCT-2021.shp
Fixing 2021-NOVIEMBRE/IMAG-SENT-NOV-2021.shp
Fixing 2018-NOV/IMAG-SENT-NOV-2018.shp
Fixing 2019-JULIO/IMAG-SENT-JUL-2019.shp
Fixing 2017-JULIO/IMAG-SENT-JUL-2017.shp
Fixing 2021-DICIEMBRE/IMAG-SENT-DIC-2021.shp
Fixing 2020-DICIEMBRE/IMAG-SENT-DIC-2020.shp
Fixing 2017-DIC/IMAG-SENT-DIC-2017.shp
Warning 1: Value of field 'layer' is not a valid UTF-8 string.
Warning 1: Value of field 'path' is not a valid UTF-8 string.
Fixing 2019-AGOSTO/IMAG-SENT-AGO-2019.shp


### Merge rasters (combinar rasters, contiguos y/o solapados, quitando las áreas negras/sin data)

In [ ]:
def merge_rasters(location):
    target_file = location + "/" + location.split("/")[-1] + "-MOSAIC-SENTINEL.tif"    
    input_rasters = list(map(lambda x: x.as_posix(), pathlib.Path(location).rglob('*.tif')))
        
    # crear mosaico si hay mas de un raster en el directorio
    if len(input_rasters) > 1:
        !gdal_merge.py -o $target_file -n 0 {" ".join(input_rasters)}

In [ ]:
base_dir = "."

for location in set(map(lambda x: x.parent.as_posix(), pathlib.Path(base_dir).rglob('*.shp'))):
    print("Merging rasters in " + location)
    merge_rasters(location)

Merging rasters in 2018-ENERO
Merging rasters in 2017-NOV
0...10...20...30...40...50...60...70...80...90...100 - done.
Merging rasters in 2021-DICIEMBRE
Merging rasters in 2021-OCTUBRE
Merging rasters in 2021-ENERO
Merging rasters in 2019-JULIO
Merging rasters in 2018-NOV
0...10...20...30...40...50...60...70...80...90...100 - done.
Merging rasters in 2017-DIC
0...10...20...30...40...50...60...70...80...90...100 - done.
Merging rasters in 2016-NOV
0...10...20...30...40...50...60...70...80...90...100 - done.
Merging rasters in 2015-SEPT
Merging rasters in 2020-DICIEMBRE
Merging rasters in 2021-NOVIEMBRE
Merging rasters in 2021-MAYO
Merging rasters in 2019-AGOSTO
Merging rasters in 2017-JULIO
0...10...20...30...40...50...60...70...80...90...100 - done.
Merging rasters in 2018-JULIO
0...10...20...30...40...50...60...70...80...90...100 - done.


### Reorganizar rasters (cambiar orden de las bandas para que esten en orden R,G,B)

In [5]:
def select_raster(location):
    tif = list(map(lambda x: x.as_posix(), pathlib.Path(location).rglob('*.tif')))
    # Elegir el raster mosaico si hay mas de un raster en el directorio
    if len(tif) > 1:
        tif = list(map(lambda x: x.as_posix(), pathlib.Path(location).rglob('*MOSAIC*.tif')))
    return tif[0]

def reorder_bands(raster_file):
    target_file = str(pathlib.PosixPath(raster_file).parent) + "/" + str(pathlib.PosixPath(raster_file).stem) + "_RGB.tif"
        
    # Reorganizar bandas
    !gdal_translate $raster_file $target_file -b 3 -b 2 -b 1

In [ ]:
for location in set(map(lambda x: select_raster(x.parent.as_posix()), pathlib.Path(base_dir).rglob('*.shp'))):
    print("Reorganizing rasters in " + location)
    reorder_bands(location)